In [1]:
func_path = 'example_data/preprocessed/func_words_preprocessed.nii.gz'
evs_path = 'example_data/preprocessed/words_events.tsv'

In [2]:
import pandas as pd

events = pd.read_csv(evs_path, sep='\t')
events.head()

,onset,duration,trial_type
0,0.0,0.8,Concrete
1,3.2,0.7,Abstract
2,6.8,0.6,Nonword
3,10.5,0.7,Nonword
4,13.2,0.7,Nonword


In [4]:
import nibabel as nib

fmri_img = nib.load(func_path)

In [5]:
import numpy as np
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix

# Assuming 'fmri_img' is your fMRI data and 'tr' is the time of repetition for your fMRI data
tr = 3.0
fmri_glm = FirstLevelModel(t_r=tr)

# Assuming 'frame_times' are the times of acquisition of your fMRI volumes
frame_times = np.arange(fmri_img.shape[-1]) * tr

# Now, we'll set up the design matrix using the events DataFrame 'events' that we loaded earlier
# We'll consider three different trial_types or conditions, i.e., 'Concrete', 'Abstract', and 'Nonword'
conditions = events['trial_type'].tolist()
onsets = events['onset'].tolist()
durations = events['duration'].tolist()

events_df = pd.DataFrame({'trial_type': conditions, 'onset': onsets, 'duration': durations})

design_matrix = make_first_level_design_matrix(frame_times, events_df, hrf_model='spm')

# Fit the GLM
fmri_glm = fmri_glm.fit(fmri_img, design_matrices=design_matrix)